In [2]:
from dimod import (
    Binary,
    BinaryQuadraticModel,
    ConstrainedQuadraticModel,
)

from dwave.system import LeapHybridCQMSampler

import pandas as pd
import numpy as np

# Define Token here to run
from credentials import TOKEN

from process_output import get_routes_from_sample, get_cost_routes, report_output, check_feasibility_sample, lazy_sanity_check

from vrp import *


import random
random.seed(7)

In [2]:
# Create a sample problem
num_destinations = 5
num_vehicles = 1
max_distance = 300

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        # Select random values that do not violate the triangle inequality
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(5,9)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

# Print the adjacency matrix
for row in cost_matrix:
    print(row)

[0, 7, 6, 8, 5, 5]
[7, 0, 9, 5, 7, 9]
[6, 9, 0, 5, 9, 6]
[8, 5, 5, 0, 5, 5]
[5, 7, 9, 5, 0, 8]
[5, 9, 6, 5, 8, 0]


### Test 1 - Single Vehicle Routing Problem

In [3]:
model = DefaultRoutingModel(num_destinations, distances=cost_matrix, num_vehicles=num_vehicles, max_distance = max_distance)
model.build_constrained_model()
print(model.cqm)

Constrained quadratic model: 30 variables, 11 constraints, 403 biases

Objective
  7*Binary('0_1_0') + 6*Binary('0_2_0') + 8*Binary('0_3_0') + 5*Binary('0_4_0') + 5*Binary('0_5_0') + 7*Binary('0_1_4') + 6*Binary('0_2_4') + 8*Binary('0_3_4') + 5*Binary('0_4_4') + 5*Binary('0_5_4') + 7*Binary('0_1_0')*Binary('0_0_1') + 6*Binary('0_2_0')*Binary('0_0_1') + 8*Binary('0_3_0')*Binary('0_0_1') + 5*Binary('0_4_0')*Binary('0_0_1') + 5*Binary('0_5_0')*Binary('0_0_1') + 9*Binary('0_2_0')*Binary('0_1_1') + 5*Binary('0_3_0')*Binary('0_1_1') + 7*Binary('0_4_0')*Binary('0_1_1') + 9*Binary('0_5_0')*Binary('0_1_1') + 7*Binary('0_0_0')*Binary('0_1_1') + 9*Binary('0_1_0')*Binary('0_2_1') + 5*Binary('0_3_0')*Binary('0_2_1') + 9*Binary('0_4_0')*Binary('0_2_1') + 6*Binary('0_5_0')*Binary('0_2_1') + 6*Binary('0_0_0')*Binary('0_2_1') + 5*Binary('0_1_0')*Binary('0_3_1') + 5*Binary('0_2_0')*Binary('0_3_1') + 5*Binary('0_4_0')*Binary('0_3_1') + 5*Binary('0_5_0')*Binary('0_3_1') + 8*Binary('0_0_0')*Binary('0_3_1')

In [4]:
feasible_sampleset = model.run_constrained_model(TOKEN)


Feasible solution found.



In [5]:
lowest_energy_sample = feasible_sampleset.lowest().first.sample

routes = get_routes_from_sample(lowest_energy_sample, num_vehicles, num_destinations)

print(f'Best route: {routes}')

print(f'Best cost: {feasible_sampleset.lowest().first.energy}')

print(f'Cost calculated: {get_cost_routes(routes, distances=cost_matrix)}')

print(f'Feasible: {check_feasibility_sample(lowest_energy_sample, num_vehicles, num_destinations, cost_matrix, max_distance, debug=True)}')

# Sanity check on cost
assert feasible_sampleset.lowest().first.energy == get_cost_routes(routes, distances=cost_matrix) * 1.0, "Cost calculations do not agree"

Best route: [[5, 2, 3, 1, 4]]
Best cost: 33.0
Cost calculated: 33.0
[False, True, True, True, True, True]
Feasible: True


In [6]:
qubit_efficient_model = BoundedPathModel(num_locations=num_destinations, distances=cost_matrix, num_vehicles=num_vehicles, max_distance = max_distance)
qubit_efficient_model.build_constrained_model()
print(qubit_efficient_model.cqm)

In bounded path
Constrained quadratic model: 30 variables, 11 constraints, 403 biases

Objective
  7*Binary('0_1_0') + 6*Binary('0_2_0') + 8*Binary('0_3_0') + 5*Binary('0_4_0') + 5*Binary('0_5_0') + 7*Binary('0_1_4') + 6*Binary('0_2_4') + 8*Binary('0_3_4') + 5*Binary('0_4_4') + 5*Binary('0_5_4') + 7*Binary('0_1_0')*Binary('0_0_1') + 6*Binary('0_2_0')*Binary('0_0_1') + 8*Binary('0_3_0')*Binary('0_0_1') + 5*Binary('0_4_0')*Binary('0_0_1') + 5*Binary('0_5_0')*Binary('0_0_1') + 9*Binary('0_2_0')*Binary('0_1_1') + 5*Binary('0_3_0')*Binary('0_1_1') + 7*Binary('0_4_0')*Binary('0_1_1') + 9*Binary('0_5_0')*Binary('0_1_1') + 7*Binary('0_0_0')*Binary('0_1_1') + 9*Binary('0_1_0')*Binary('0_2_1') + 5*Binary('0_3_0')*Binary('0_2_1') + 9*Binary('0_4_0')*Binary('0_2_1') + 6*Binary('0_5_0')*Binary('0_2_1') + 6*Binary('0_0_0')*Binary('0_2_1') + 5*Binary('0_1_0')*Binary('0_3_1') + 5*Binary('0_2_0')*Binary('0_3_1') + 5*Binary('0_4_0')*Binary('0_3_1') + 5*Binary('0_5_0')*Binary('0_3_1') + 8*Binary('0_0_0')

In [7]:
efficient_feasible_sampleset = qubit_efficient_model.run_constrained_model(TOKEN)


Feasible solution found.



In [8]:
lowest_energy_sample = efficient_feasible_sampleset.lowest().first.sample

routes = get_routes_from_sample(lowest_energy_sample, num_vehicles, num_destinations)

print(f'Best route: {routes}')

print(f'Best cost: {efficient_feasible_sampleset.lowest().first.energy}')

print(f'Cost calculated: {get_cost_routes(routes, distances=cost_matrix)}')

print(f'Feasible: {check_feasibility_sample(lowest_energy_sample, num_vehicles, num_destinations, cost_matrix, max_distance, debug=True)}')

# Sanity check on cost
assert efficient_feasible_sampleset.lowest().first.energy == get_cost_routes(routes, distances=cost_matrix) * 1.0, "Cost calculations do not agree"

Best route: [[5, 2, 3, 1, 4]]
Best cost: 33.0
Cost calculated: 33.0
[False, True, True, True, True, True]
Feasible: True


In [9]:
# Lazy sanity check
r = []
for i in range(1, 6):
    r.append(i)

min = 300
best_route = r
# Randomly sample a lot of options
for i in range(10000):
    random.shuffle(r)

    cost = get_cost_routes([r], cost_matrix)
    if cost < min:
        min = cost
        best_routes = [r]
print(min)
print(best_routes)

33.0
[[5, 1, 4, 3, 2]]


### Test 2: Capacitated Multiple Vehicle Routing Problem

In [10]:
model2 = DefaultRoutingModel(num_destinations, cost_matrix, num_vehicles=2, max_distance=30)
model2.build_constrained_model()
print(model2.cqm)

Constrained quadratic model: 60 variables, 17 constraints, 806 biases

Objective
  7*Binary('0_1_0') + 6*Binary('0_2_0') + 8*Binary('0_3_0') + 5*Binary('0_4_0') + 5*Binary('0_5_0') + 7*Binary('1_1_0') + 6*Binary('1_2_0') + 8*Binary('1_3_0') + 5*Binary('1_4_0') + 5*Binary('1_5_0') + 7*Binary('0_1_4') + 6*Binary('0_2_4') + 8*Binary('0_3_4') + 5*Binary('0_4_4') + 5*Binary('0_5_4') + 7*Binary('1_1_4') + 6*Binary('1_2_4') + 8*Binary('1_3_4') + 5*Binary('1_4_4') + 5*Binary('1_5_4') + 7*Binary('0_1_0')*Binary('0_0_1') + 6*Binary('0_2_0')*Binary('0_0_1') + 8*Binary('0_3_0')*Binary('0_0_1') + 5*Binary('0_4_0')*Binary('0_0_1') + 5*Binary('0_5_0')*Binary('0_0_1') + 9*Binary('0_2_0')*Binary('0_1_1') + 5*Binary('0_3_0')*Binary('0_1_1') + 7*Binary('0_4_0')*Binary('0_1_1') + 9*Binary('0_5_0')*Binary('0_1_1') + 7*Binary('0_0_0')*Binary('0_1_1') + 9*Binary('0_1_0')*Binary('0_2_1') + 5*Binary('0_3_0')*Binary('0_2_1') + 9*Binary('0_4_0')*Binary('0_2_1') + 6*Binary('0_5_0')*Binary('0_2_1') + 6*Binary('0_0

In [11]:
feasible_sampleset2 = model2.run_constrained_model(token=TOKEN)


Feasible solution found.



In [20]:
def get_cost_routes(paths, distances):
    costs = []

    num_vertices = len(distances)

    for path in paths:

        cost = 0

        if len(path) > 0:

            # Cost in middle of path
            for i in range(len(path) - 1):
                cost += distances[path[i]][path[i+1]]

            # Cost of first choice
            cost += distances[0][path[0]]

            # Cost of last choice
            cost += distances[path[len(path) - 1]][0]

        costs.append(cost * 1.0)

    return costs

In [13]:
lowest_energy_sample2 = feasible_sampleset2.lowest().first.sample

routes2 = get_routes_from_sample(lowest_energy_sample2, num_vehicles=2, num_steps=num_destinations)

print(f'Routes: {routes2}')

print(f'Best cost: {feasible_sampleset2.lowest().first.energy}')

Routes: [[0, 0, 0, 0, 4], [5, 2, 3, 1, 0]]
Best cost: 38.0


In [14]:
nmodel2 = BoundedPathModel(num_destinations, cost_matrix, num_vehicles=2, max_distance=30)
nmodel2.build_constrained_model()
nmodel2.path_lengths

In bounded path


[5, 3]

In [15]:
nfeasible_sampleset2 = nmodel2.run_constrained_model(token=TOKEN)


Feasible solution found.



In [16]:
nlowest_energy_sample2 = nfeasible_sampleset2.lowest().first.sample

nroutes2 = get_routes_from_sample(nlowest_energy_sample2, num_vehicles=2, num_steps=num_destinations)

print(f'Routes: {nroutes2}')

print(f'Best cost: {nfeasible_sampleset2.lowest().first.energy}')

Routes: [[0, 5, 3, 4, -1], [0, 0, 0, -1, -1]]
Best cost: 25.0


In [17]:
print(nlowest_energy_sample2)

{'0_0_0': 1.0, '0_0_1': 0.0, '0_0_2': 0.0, '0_0_3': 0.0, '0_0_4': 0.0, '0_1_0': 0.0, '0_1_1': 0.0, '0_1_2': 0.0, '0_1_3': 1.0, '0_1_4': 0.0, '0_2_0': 0.0, '0_2_1': 0.0, '0_2_2': 0.0, '0_2_3': 1.0, '0_2_4': 0.0, '0_3_0': 0.0, '0_3_1': 0.0, '0_3_2': 1.0, '0_3_3': 0.0, '0_3_4': 0.0, '0_4_0': 0.0, '0_4_1': 0.0, '0_4_2': 0.0, '0_4_3': 1.0, '0_4_4': 0.0, '0_5_0': 0.0, '0_5_1': 1.0, '0_5_2': 0.0, '0_5_3': 0.0, '0_5_4': 0.0, '1_0_0': 1.0, '1_0_1': 1.0, '1_0_2': 1.0, '1_1_0': 0.0, '1_1_1': 0.0, '1_1_2': 0.0, '1_2_0': 0.0, '1_2_1': 0.0, '1_2_2': 0.0, '1_3_0': 0.0, '1_3_1': 0.0, '1_3_2': 0.0, '1_4_0': 0.0, '1_4_1': 0.0, '1_4_2': 0.0, '1_5_0': 0.0, '1_5_1': 0.0, '1_5_2': 0.0}


In [18]:
# Lazy sanity check
r = []
for i in range(1, 5+1):
    r.append(i)

min = 300
best_route = r
# Randomly sample a lot of options
for i in range(10000):
    random.shuffle(r)

    r1 = r[:2]
    r2 = r[2:]

    cost = get_cost_routes([r1, r2], cost_matrix)
    if cost < min:
        min = cost
        best_routes = [r1, r2]
print(min)
print(best_routes)

39.0
[[2, 5], [1, 3, 4]]


In [20]:
val, route = lazy_sanity_check(num_destinations, distances=cost_matrix, num_vehicles=2, max_distance=30)
print(val)
print(route)

39.0
[[5, 2], [4, 3, 1]]


### Larger Test

Try to generate a minimal example on which the solver fails

In [3]:
# Create a sample problem
num_destinations = 15
num_vehicles = 3
max_distance = 60

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        # Select random values that do not violate the triangle inequality
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(5,9)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

# Print the adjacency matrix
for row in cost_matrix:
    print(row)

[0, 7, 6, 8, 5, 5, 9, 5, 7, 9, 5, 9, 6, 5, 5, 8]
[7, 0, 8, 5, 6, 5, 9, 8, 5, 9, 5, 6, 9, 5, 9, 9]
[6, 8, 0, 8, 5, 6, 5, 9, 6, 7, 8, 6, 9, 5, 9, 7]
[8, 5, 8, 0, 9, 6, 5, 9, 9, 6, 7, 5, 9, 5, 9, 5]
[5, 6, 5, 9, 0, 9, 6, 8, 9, 8, 7, 8, 9, 8, 7, 7]
[5, 5, 6, 6, 9, 0, 6, 6, 6, 5, 9, 7, 9, 8, 7, 8]
[9, 9, 5, 5, 6, 6, 0, 7, 9, 5, 5, 9, 8, 6, 7, 6]
[5, 8, 9, 9, 8, 6, 7, 0, 8, 8, 5, 5, 9, 9, 7, 7]
[7, 5, 6, 9, 9, 6, 9, 8, 0, 7, 9, 8, 9, 8, 5, 5]
[9, 9, 7, 6, 8, 5, 5, 8, 7, 0, 7, 8, 5, 5, 7, 9]
[5, 5, 8, 7, 7, 9, 5, 5, 9, 7, 0, 8, 7, 8, 7, 5]
[9, 6, 6, 5, 8, 7, 9, 5, 8, 8, 8, 0, 8, 7, 6, 9]
[6, 9, 9, 9, 9, 9, 8, 9, 9, 5, 7, 8, 0, 5, 8, 5]
[5, 5, 5, 5, 8, 8, 6, 9, 8, 5, 8, 7, 5, 0, 6, 7]
[5, 9, 9, 9, 7, 7, 7, 7, 5, 7, 7, 6, 8, 6, 0, 6]
[8, 9, 7, 5, 7, 8, 6, 7, 5, 9, 5, 9, 5, 7, 6, 0]


In [22]:
model3 = DefaultRoutingModel(num_destinations, cost_matrix, num_vehicles=num_vehicles, max_distance=max_distance)
model3.build_constrained_model()
feasible_sampleset3 = model3.run_constrained_model(token=TOKEN)


Feasible solution found.



In [23]:
print(model3.cqm)

Constrained quadratic model: 720 variables, 63 constraints, 24339 biases

Objective
  7*Binary('0_1_0') + 9*Binary('0_2_0') + 9*Binary('0_3_0') + 5*Binary('0_4_0') + 5*Binary('0_5_0') + 7*Binary('0_6_0') + 7*Binary('0_7_0') + 5*Binary('0_8_0') + 5*Binary('0_9_0') + 8*Binary('0_10_0') + 6*Binary('0_11_0') + 8*Binary('0_12_0') + 9*Binary('0_13_0') + 8*Binary('0_14_0') + 9*Binary('0_15_0') + 7*Binary('1_1_0') + 9*Binary('1_2_0') + 9*Binary('1_3_0') + 5*Binary('1_4_0') + 5*Binary('1_5_0') + 7*Binary('1_6_0') + 7*Binary('1_7_0') + 5*Binary('1_8_0') + 5*Binary('1_9_0') + 8*Binary('1_10_0') + 6*Binary('1_11_0') + 8*Binary('1_12_0') + 9*Binary('1_13_0') + 8*Binary('1_14_0') + 9*Binary('1_15_0') + 7*Binary('2_1_0') + 9*Binary('2_2_0') + 9*Binary('2_3_0') + 5*Binary('2_4_0') + 5*Binary('2_5_0') + 7*Binary('2_6_0') + 7*Binary('2_7_0') + 5*Binary('2_8_0') + 5*Binary('2_9_0') + 8*Binary('2_10_0') + 6*Binary('2_11_0') + 8*Binary('2_12_0') + 9*Binary('2_13_0') + 8*Binary('2_14_0') + 9*Binary('2_15_0'

In [24]:
lowest_energy_sample3 = feasible_sampleset3.lowest().first.sample

routes3 = get_routes_from_sample(lowest_energy_sample3, num_vehicles=num_vehicles, num_steps=num_destinations)

print(f'Routes: {routes3}')

print(f'Best cost: {feasible_sampleset3.lowest().first.energy}')

Routes: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 11, 6, 12, 5, 14, 1, 7, 8, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 9, 13, 3, 2, 10, 15, 4]]
Best cost: 91.0


In [25]:
val, route = lazy_sanity_check(num_destinations, distances=cost_matrix, num_vehicles=num_vehicles, max_distance=max_distance)
print(val)
print(route)
print(num_destinations)

print(feasible_sampleset3)

110.0
[[11, 12, 14, 5, 4], [9, 10, 2, 1, 8], [3, 13, 15, 7, 6]]
15
   0_0_0 0_0_1 0_0_10 0_0_11 0_0_12 0_0_13 0_0_14 ... 2_9_9 energy num_oc. ...
0    1.0   1.0    1.0    1.0    1.0    1.0    1.0 ...   0.0   91.0       1 ...
3    1.0   1.0    1.0    1.0    1.0    1.0    1.0 ...   0.0   91.0       1 ...
4    1.0   1.0    1.0    1.0    1.0    1.0    1.0 ...   0.0   91.0       1 ...
8    1.0   1.0    0.0    0.0    0.0    1.0    1.0 ...   0.0   91.0       1 ...
10   1.0   1.0    0.0    0.0    0.0    1.0    1.0 ...   0.0   91.0       1 ...
15   1.0   1.0    0.0    0.0    0.0    1.0    1.0 ...   0.0   91.0       1 ...
19   1.0   1.0    1.0    1.0    1.0    1.0    1.0 ...   0.0   91.0       1 ...
22   1.0   1.0    1.0    1.0    1.0    1.0    1.0 ...   0.0   91.0       1 ...
29   1.0   1.0    1.0    1.0    1.0    1.0    1.0 ...   0.0   91.0       1 ...
30   1.0   1.0    0.0    0.0    0.0    1.0    1.0 ...   0.0   91.0       1 ...
5    1.0   1.0    0.0    0.0    1.0    1.0    1.0 ...   0.0   92

### Compare to results with a reduced qubit representation

In [ ]:
qubit_efficient_model = BoundedPathModel(num_locations=num_destinations, distances=cost_matrix, num_vehicles=num_vehicles, max_distance=max_distance)
qubit_efficient_model.build_constrained_model()

In [8]:
feasible_sampleset4 = qubit_efficient_model.run_constrained_model(token=TOKEN)


Feasible solution found.



In [10]:
# Consider the size of the problem
print(qubit_efficient_model.cqm)

Constrained quadratic model: 448 variables, 46 constraints, 14564 biases

Objective
  7*Binary('0_1_0') + 6*Binary('0_2_0') + 8*Binary('0_3_0') + 5*Binary('0_4_0') + 5*Binary('0_5_0') + 9*Binary('0_6_0') + 5*Binary('0_7_0') + 7*Binary('0_8_0') + 9*Binary('0_9_0') + 5*Binary('0_10_0') + 9*Binary('0_11_0') + 6*Binary('0_12_0') + 5*Binary('0_13_0') + 5*Binary('0_14_0') + 8*Binary('0_15_0') + 7*Binary('1_1_0') + 6*Binary('1_2_0') + 8*Binary('1_3_0') + 5*Binary('1_4_0') + 5*Binary('1_5_0') + 9*Binary('1_6_0') + 5*Binary('1_7_0') + 7*Binary('1_8_0') + 9*Binary('1_9_0') + 5*Binary('1_10_0') + 9*Binary('1_11_0') + 6*Binary('1_12_0') + 5*Binary('1_13_0') + 5*Binary('1_14_0') + 8*Binary('1_15_0') + 7*Binary('2_1_0') + 6*Binary('2_2_0') + 8*Binary('2_3_0') + 5*Binary('2_4_0') + 5*Binary('2_5_0') + 9*Binary('2_6_0') + 5*Binary('2_7_0') + 7*Binary('2_8_0') + 9*Binary('2_9_0') + 5*Binary('2_10_0') + 9*Binary('2_11_0') + 6*Binary('2_12_0') + 5*Binary('2_13_0') + 5*Binary('2_14_0') + 8*Binary('2_15_0'

In [11]:
def parse_string(input_string):
    return list(map(int, input_string.split('_')))

def get_routes_from_sample(sample, num_vehicles, num_steps, custom_path_lengths=None):
    """Builds a set of routes from the sample returned."""

    if custom_path_lengths:
        routes = []
        for length in custom_path_lengths:
            routes.extend([[-1]*length])
    else:
        routes =  [[-1]*num_steps for _ in range(num_vehicles)]

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            routes[vehicle][step] = vertex
    
    return routes

In [22]:
def check_feasibility_sample(sample, num_vehicles, num_steps, distances, max_distance, debug=False):

    routes =  [[-1]*num_steps for _ in range(num_vehicles)]

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            if routes[vehicle][step] == -1:
                routes[vehicle][step] = vertex
            else:
                # Location visited multiple times
                if debug:
                    print(f"Violated constraint that vehicle {vehicle} can not be at multiple locations at once (visits {vertex} and {routes[vehicle][step]} at step {step})")
                return False
            
    print(f"Feasibility check routes: {routes}")
            
    # Next, check that everything is visited
    visited = [False]*(num_steps+1)

    for route in routes:
        for step in route:
            if step != -1:
                visited[step] = True

    if debug: print(visited)
    
    # Check all locations except depot since there is an implicit start and end visit
    for i in range(1, len(visited)):
        if not visited[i]:
            if debug:
                print(f"Violated constraint that location {i} must be visited")
            return False

    # Finally, check that capacities are respected
    for idx, route in enumerate(routes):
        cost = 0

        if route:
            cost += distances[num_steps][route[0]]
            cost += distances[route[-1]][num_steps]

            for i in range(num_steps - 1):
                cost += distances[route[i]][route[i+1]]

        if cost > max_distance:
            if debug:
                print(f"Violated constraint that vehicle {idx} must travel under {max_distance} distance (drives {cost})")
            return False

    return True

In [24]:
lowest_energy_sample4 = feasible_sampleset4.lowest().first.sample

print(lowest_energy_sample4)

# print(feasible_sampleset4)

print()

routes4 = get_routes_from_sample(lowest_energy_sample4, num_vehicles=num_vehicles, num_steps=num_destinations, custom_path_lengths=qubit_efficient_model.path_lengths)

print(f'Routes: {routes4}')

print()

print(f'Best cost: {feasible_sampleset4.lowest().first.energy}')

print()

print(check_feasibility_sample(lowest_energy_sample4, num_vehicles, num_destinations, cost_matrix, max_distance, debug=True))

print()

print(qubit_efficient_model.path_lengths)

print()

print(get_cost_routes(routes4, cost_matrix))

{'0_0_0': 1.0, '0_0_1': 1.0, '0_0_10': 0.0, '0_0_11': 0.0, '0_0_12': 0.0, '0_0_13': 1.0, '0_0_14': 1.0, '0_0_2': 1.0, '0_0_3': 1.0, '0_0_4': 1.0, '0_0_5': 0.0, '0_0_6': 0.0, '0_0_7': 0.0, '0_0_8': 0.0, '0_0_9': 0.0, '0_10_0': 0.0, '0_10_1': 0.0, '0_10_10': 0.0, '0_10_11': 0.0, '0_10_12': 0.0, '0_10_13': 0.0, '0_10_14': 0.0, '0_10_2': 0.0, '0_10_3': 0.0, '0_10_4': 0.0, '0_10_5': 0.0, '0_10_6': 0.0, '0_10_7': 0.0, '0_10_8': 0.0, '0_10_9': 0.0, '0_11_0': 0.0, '0_11_1': 0.0, '0_11_10': 0.0, '0_11_11': 1.0, '0_11_12': 0.0, '0_11_13': 0.0, '0_11_14': 0.0, '0_11_2': 0.0, '0_11_3': 0.0, '0_11_4': 0.0, '0_11_5': 0.0, '0_11_6': 0.0, '0_11_7': 0.0, '0_11_8': 0.0, '0_11_9': 0.0, '0_12_0': 0.0, '0_12_1': 0.0, '0_12_10': 0.0, '0_12_11': 0.0, '0_12_12': 0.0, '0_12_13': 0.0, '0_12_14': 0.0, '0_12_2': 0.0, '0_12_3': 0.0, '0_12_4': 0.0, '0_12_5': 0.0, '0_12_6': 0.0, '0_12_7': 0.0, '0_12_8': 1.0, '0_12_9': 0.0, '0_13_0': 0.0, '0_13_1': 0.0, '0_13_10': 0.0, '0_13_11': 0.0, '0_13_12': 0.0, '0_13_13': 0.0, 

In [21]:
test = feasible_sampleset4.lowest().first.sample
test_sample = feasible_sampleset4.slice(87, 88)
lowest_energy_sample4 = test_sample.first.sample

print(lowest_energy_sample4)

# print(feasible_sampleset4)

print()

routes4 = get_routes_from_sample(lowest_energy_sample4, num_vehicles=num_vehicles, num_steps=num_destinations, custom_path_lengths=qubit_efficient_model.path_lengths)

print(f'Routes: {routes4}')

print()

print(f'Best cost: {test_sample.first.energy}')

print()

print(check_feasibility_sample(lowest_energy_sample4, num_vehicles, num_destinations, cost_matrix, max_distance, debug=True))

print()

print(qubit_efficient_model.path_lengths)

print()

print(get_cost_routes(routes4, cost_matrix))

{'0_0_0': 1.0, '0_0_1': 1.0, '0_0_10': 0.0, '0_0_11': 0.0, '0_0_12': 0.0, '0_0_13': 1.0, '0_0_14': 1.0, '0_0_2': 0.0, '0_0_3': 1.0, '0_0_4': 0.0, '0_0_5': 1.0, '0_0_6': 0.0, '0_0_7': 1.0, '0_0_8': 1.0, '0_0_9': 1.0, '0_10_0': 0.0, '0_10_1': 0.0, '0_10_10': 0.0, '0_10_11': 0.0, '0_10_12': 0.0, '0_10_13': 0.0, '0_10_14': 0.0, '0_10_2': 0.0, '0_10_3': 0.0, '0_10_4': 0.0, '0_10_5': 0.0, '0_10_6': 0.0, '0_10_7': 0.0, '0_10_8': 0.0, '0_10_9': 0.0, '0_11_0': 0.0, '0_11_1': 0.0, '0_11_10': 0.0, '0_11_11': 0.0, '0_11_12': 0.0, '0_11_13': 0.0, '0_11_14': 0.0, '0_11_2': 0.0, '0_11_3': 0.0, '0_11_4': 0.0, '0_11_5': 0.0, '0_11_6': 0.0, '0_11_7': 0.0, '0_11_8': 0.0, '0_11_9': 0.0, '0_12_0': 0.0, '0_12_1': 0.0, '0_12_10': 1.0, '0_12_11': 0.0, '0_12_12': 0.0, '0_12_13': 0.0, '0_12_14': 0.0, '0_12_2': 0.0, '0_12_3': 0.0, '0_12_4': 0.0, '0_12_5': 0.0, '0_12_6': 0.0, '0_12_7': 0.0, '0_12_8': 0.0, '0_12_9': 0.0, '0_13_0': 0.0, '0_13_1': 0.0, '0_13_10': 0.0, '0_13_11': 0.0, '0_13_12': 0.0, '0_13_13': 0.0, 

In [ ]:
test_model = BoundedPathModel(num_locations=num_destinations, distances=cost_matrix, num_vehicles=num_vehicles, max_distance=max_distance)
test_model.build_constrained_model()

for constraint in test_model.cqm.constraints:
    print(constraint)